In [1]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [2]:
%cd ..

/home/zaccharie/workspace/fastmri-reproducible-benchmark


In [3]:
import time
import os.path as op

import numpy as np
import torch
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm_notebook

from pdnet_crop import PDNetCrop
from data_torch import MaskedUntouched2DDataset, MaskedUntouched2DAllLoadedDataset
from torch_training import fit_torch, torch_psnr

Using TensorFlow backend.


In [4]:
# paths
# train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
# val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
# test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

val_path = '../singlecoil_val/'
train_path = val_path
test_path = '../singlecoil_test/'

n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [5]:
# data loader
# generators
AF = 4
# train_gen = MaskedUntouched2DAllLoadedDataset(train_path, af=AF, inner_slices=1)
train_gen = MaskedUntouched2DDataset(train_path, af=AF, inner_slices=1)
val_gen = MaskedUntouched2DDataset(val_path, af=AF)

train_loader = DataLoader(
    dataset=train_gen,
    batch_size=1,
    shuffle=False,
    num_workers=10,
    pin_memory=True,
)

In [6]:
run_params = {
    'n_primal': 2,
    'n_dual': 2,
    'n_iter': 10,
    'n_filters': 32,
    'primal_only': False,
}
n_epochs = 500
run_id = f'pdnet_torch_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
log_dir = op.join('logs', run_id)
print(run_id)

pdnet_torch_af4_1568296527


In [7]:
model = PDNetCrop(**run_params)
optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-7)
writer = SummaryWriter(log_dir=log_dir)

# model.cuda();

In [8]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [9]:
params

249040

In [10]:
def overfit_epoch(model, data, optimizer, device):
    model.train()
    kspace, mask, image_gt = data
    kspace = kspace[0] * 1e6
    mask = mask[0]
    image_gt = image_gt[0] * 1e6
    kspace = kspace.to(device)
    mask = mask.to(device)
    image_gt = image_gt.to(device)
    image_pred = model(kspace, mask)


    loss = F.l1_loss(image_pred, image_gt)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    psnr = torch_psnr(image_pred, image_gt)
    print('Training PSNR:', psnr)

In [ ]:
# %%time
i, data = next(enumerate(train_loader))
for _ in tqdm_notebook(range(500)):
    overfit_epoch(model, data, optimizer, 'cpu')

Training PSNR: tensor(15.4876, grad_fn=<MulBackward0>)
Training PSNR: tensor(16.2738, grad_fn=<MulBackward0>)
Training PSNR: tensor(17.8109, grad_fn=<MulBackward0>)
Training PSNR: tensor(19.3993, grad_fn=<MulBackward0>)
Training PSNR: tensor(19.3083, grad_fn=<MulBackward0>)
Training PSNR: tensor(22.6661, grad_fn=<MulBackward0>)
Training PSNR: tensor(22.4735, grad_fn=<MulBackward0>)
Training PSNR: tensor(21.9442, grad_fn=<MulBackward0>)
Training PSNR: tensor(22.3033, grad_fn=<MulBackward0>)
Training PSNR: tensor(22.9027, grad_fn=<MulBackward0>)
Training PSNR: tensor(23.0537, grad_fn=<MulBackward0>)
Training PSNR: tensor(23.0528, grad_fn=<MulBackward0>)
Training PSNR: tensor(23.5714, grad_fn=<MulBackward0>)
Training PSNR: tensor(24.1247, grad_fn=<MulBackward0>)
Training PSNR: tensor(24.2099, grad_fn=<MulBackward0>)
Training PSNR: tensor(24.1388, grad_fn=<MulBackward0>)
Training PSNR: tensor(24.2167, grad_fn=<MulBackward0>)
Training PSNR: tensor(24.3787, grad_fn=<MulBackward0>)
Training P

In [ ]:
# %%time
# train_gen.filenames = train_gen.filenames[:10]
# val_gen.filenames = val_gen.filenames[:1]
# train_loader = DataLoader(
#     dataset=train_gen,
#     batch_size=1,
#     shuffle=False,
#     num_workers=10,
#     pin_memory=True,
# )
# val_loader = DataLoader(
#     dataset=val_gen,
#     batch_size=1,
# #     num_workers=35,
#     pin_memory=True,
#     shuffle=False,
# )
# fit_torch(
#     model, 
#     train_loader, 
#     val_loader, 
#     n_epochs,
#     writer, 
#     optimizer, 
#     chkpt_path, 
#     run_id=run_id, 
#     device='cuda', 
#     save_freq=500, 
#     tqdm_wrapper=tqdm_notebook,
# )